## Lab 10_11: AE2: Wypełnianie koła prostokątami (2 tygodnie, 6 punktów, uwaga na podział punktów na terminy!!)

Rozwiązać wariant problemu znanego w literaturze jako cutting stock problem.

Mamy dane koło o promieniu r oraz zbiór dostępnych prostokątów zadanych przez trzy liczby: wysokość, szerokość i wartość. Celem jest ułożenie prostokątów w kole tak, aby zmaksymalizować sumę ich wartości, spełniając następujące warunki:  
- boki wszystkich prostokątów były równoległe do osi układu,  
- wnętrza prostokątów nie miały części wspólnej (intuicyjnie: prostokąty nie nachodzą na siebie, ale mogą się stykać bokami),  
- każdy prostokąt można wstawić dowolnie wiele razy.  

Zaproponować:
- kodowanie problemu, 
- operator mutacji, 
- krzyżowania  

oraz zaimplementować algorytm genetyczny rozwiązujący powyższy problem.

Przetestować rozwiązanie na danych dostarczonych w folderze `data/cutting`. Struktura pliku: w nazwie pliku jest podany promień koła, plik jest w formacie csv: 
- pierwsza kolumna to szerokość prostokąta, 
- druga to wysokość, 
- trzecia to wartość prostokąta.

## TODO:
- present the proposed solution
- implement genome encoding
- implement mutation operator
- implement crossover operator
- implement fitness function
- implement a genetic algorithm

## Proposed solution

Given a list of rectangle IDs, the fitness function will create columns of maximum height, one for each rectangle ID. Size of each column will be such, that the width is the same as the rectangle width and height will be computed assuming that each column will be right next to the previous one and in the first column there will be only 1 rectangle.

### Genome encoding

A list of integers of varying length representing the rectangle IDs. The length of the list is the number of columns in the final solution. Maximum length of the list is the maximum number of columns calculated by using circle diameter and the smallest rectangle.

### Fitness function

If at least 1 rectangle is placed in each column, the fitness function will return the sum of the rectangle values.

Otherwise, the fitness function will return the sum of the rectangle values that are placed at least once in the circle minus the sum of the rectangle values that are not placed in the column (1 for each ID).

### Mutation operator

#### Add a column of rectangles

Select random place in the genome and add a rectangle ID to the list of IDs, not necessarily at the end.

Probability of adding a rectangle ID is higher if sum of column widths is much smaller than the circle diameter.

Probability of adding a rectangle ID is lower if sum of column widths is close to or greater than the circle diameter.

#### Remove a column of rectangle

Select random ID from the list of IDs (genome) and remove it. This will effectively remove the whole column made of rectangles with the selected ID.
 
Probability of removing a rectangle ID is higher if sum of column widths is greater than the circle diameter.

Probability of removing a rectangle ID is lower if sum of column widths is close to or smaller than the circle diameter.


### Crossover operator

Single point crossover between two genomes. Each is a list of integers representing the rectangle IDs.

### Population creation

The population is a list of genomes. Each genome is a list of integers representing the rectangle IDs. Each genome will be created by first choosing a random number of columns and then choosing random IDs from the list of IDs. 

Before selecting next ID, a sum of column widths is calculated. The next ID is selected from the list of rectangle IDs that added to the sum of column widths would not make it than the circle diameter.